# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application. 

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
- Task 6: Visibility Tooling
- Task 7: RAG Evaluation Using GPT-4

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/PvlaIUO.png" />

## Task 1: Imports and Utility 

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
%load_ext autoreload
%autoreload 2

!pip install -q -U numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [3]:
!pip install PyPDF2

In [6]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter, PDFFileLoader
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [5]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with. 

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format. 

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [9]:
#text_loader = TextFileLoader("data/KingLear.txt")
#documents = text_loader.load_documents()

#use our new pdf loader
pdf_loader = PDFFileLoader("data/king-lear_PDF_FolgerShakespeare.pdf")
documents = pdf_loader.load_documents()


In [10]:
print(documents[0][:10])
print(len(documents[0]))

Get even m
204681


### Splitting Text Into Chunks

As we can see, there is one document - and it's the entire text of Frakenstein

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM. 

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length. 

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following: 
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

Let's take a peek visually at what we're doing here - and why it might be useful:

<img src="https://i.imgur.com/rtM6Ci6.png" />

As you can see (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [12]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

256

Let's take a look at some of the documents we've managed to split.

In [14]:
# split_documents[0:1]
split_documents[0:1]

['Get even mor e from the Folger\nYou can get your own copy of this text to keep. Purchase a full copy\nto get the text, plus explanatory notes, illustrations, and more.\n Buy a copyFolger Shakespeare Library\nhttps://shakespeare.folger.edu/\nFront\nMatterFrom the Director of the Folger Shakespeare\nLibrary\nTextual Introduction\nSynopsis\nCharacters in the Play\nACT 1Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nACT 2Scene 1\nScene 2\nScene 3\nScene 4\nACT 3Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nScene 6\nScene 7\nACT 4Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nScene 6\nScene 7\nACT 5Scene 1\nScene 2\nScene 3Contents\nMichael Witmore\nDirector, Folger Shakespeare LibraryIt is hard to imagine a world without Shakespeare. Since their\ncomposition four hundred years ago, Shakespeare’s plays and poems\nhave traveled the globe, inviting those who see and read his works to\nmake them their own.\nReaders of the New Folger Editions are part of this ongoing process\nof “taking up Shakesp

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format. 

Loosely, this means turning the text into numbers. 

There are plenty of resources that talk about this process in great detail - I'll leave this [blog](https://txt.cohere.com/sentence-word-embeddings/) from Cohere.AI as a resource if you want to deep dive a bit. 

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

While this is all baked into 1 call - let's look at some of the code that powers this process:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model. 

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓QUESTION:

Though 1536 is the default dimension of `text-embedding-3-small`, you can pass a dimension parameter and shorten the vector - what method does OpenAI use to achieve this shortening?

#### ANSWER:

OpenAI uses principal component analysis to reduce the number of dimensions.  PCA takes a number of vectors or features and reduces the dimensionality to the specified dimension parameter while preserving as much of the variance (information) as possible.  It uses a bunch of math to do this :D and is a well known concept from pre-LLM machine learning.

There is a chart at this link : https://openai.com/blog/new-embedding-models-and-api-updates

that shows by reducing the dimensions from 1536 to 512 we lose less than 1% on the average MTEB score.  62.3% to 61.6%.  One would think it would be much more!


In [11]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [15]:
import time

vector_db = VectorDatabase()
astart_time = time.time()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))
aend_time = time.time()
aduration = aend_time - astart_time
print(f"Async version took {aduration} seconds")

Async version took 1.6505680084228516 seconds


In [12]:
vector_db = VectorDatabase()
start_time = time.time()
vector_db = vector_db.build_from_list(split_documents)
end_time = time.time()
duration = end_time - start_time
print(f"Normal version took {duration} seconds")

Normal version took 1.2558674812316895 seconds


#### ❓QUESTION:

What advantage does an async implementation provide for us?

#### ANSWER:

The advantage is other parts of our program can continue running while the async code completes.  The classic example, in the context of a thick application, is you want your long running processing code on a different thread than your GUI code, so your program does not freeze while you wait for processing.

Network operations like above can take a few seconds or longer to complete and we don't want our program to lock up while it is running.  The Async library gives us a conveniant way of implementing this behavior.

Initially I thought it was sending each element of our string array seperately to OpenAI for processing, but after I implemented a synchronous version of the code and saw it took the same amount of time, I dug deeper into the code and saw it sends over the entire list at once.


So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus. 

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [16]:
vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)

[('s your servant Caius?\n \n He’s a good fellow, I can tell you that.\n He’ll strike and quickly too. He’s dead and rotten.\n \n No, my good lord, I am the very man—FTLN 3568\nFTLN 3569\nFTLN 3570 320\nFTLN 3571\nFTLN 3572\nFTLN 3573\nFTLN 3574\nFTLN 3575 325\nFTLN 3576\nFTLN 3577\nFTLN 3578\nFTLN 3579\nFTLN 3580 330\nFTLN 3581\nFTLN 3582\nFTLN 3583\nFTLN 3584\nFTLN 3585 335\nFTLN 3586\nFTLN 3587\nFTLN 3588\nFTLN 3589\nFTLN 3590 340\nFTLN 3591\nFTLN 3592\nFTLN 3593\nFTLN 3594\nFTLN 3595 345\n259 King LearACT 5. SC. 3\nLEAR\nKENT\nLEAR\nKENT\nLEAR\nALBANY\nEDGAR\nMESSENGER\nALBANY\nLEAR\nHe dies.  I’ll see that straight.\n \n That from your first of difference and decay\n Have followed your sad steps.\n   You are welcome\n hither.\n \n Nor no man else. All’s cheerless, dark, and deadly.\n Your eldest daughters have fordone themselves,\n And desperately are dead.\n   Ay, so I think.\n \n He knows not what he says, and vain is it\n That we present us to him.\n   Very bootless.\nEnter a M

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work. 

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)): 

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this: 

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts. 

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓QUESTION:

How could we ensure our LLM always responded the same way to our inputs?

#### ANSWER:

Generally, make sure your system prompt is the same.

Specifically, you could make use of the seed input parameter, the system_fingerprint response parameter, temperature (turn it down to 0) parameter, or top_p parameter all found here : https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages

These are all different mechanisms for trying to force the LLM to give you the same response to the same query.

### Creating and Prompting OpenAI's `gpt-3.5-turbo`!

Let's tie all these together and use it to prompt `gpt-3.5-turbo`!

In [18]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [19]:
print(response)

The best way to write a loop in Python is to use a for loop if you know the exact number of iterations needed, or a while loop if the number of iterations is dependent on a certain condition. Here is an example of each:

For loop:
```python
for i in range(5):
    print(i)
```

While loop:
```python
i = 0
while i < 5:
    print(i)
    i += 1
```

Remember to make sure your loop has a clear exit condition to prevent infinite loops. Happy coding!


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [20]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query. 

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)
        
        return {"response" : self.llm.run([formatted_user_prompt, formatted_system_prompt]), "context" : context_list}

#### ❓QUESTION:

What changes could you make that would encourage the LLM to have a more well thought out and verbose response?

What is this method called?

#### ANSWER:

You could write a better prompt engineered system prompt, such as "To ensure a comprehensive and detailed response, please review relevant data sources before generating the answer. Focus on gathering a wide range of facts, examples, and perspectives related to the query. Synthesize this information to provide a verbose and well-rounded response that not only answers the direct question but also offers contextual understanding, historical relevance, and potential implications. Aim to address any related subtopics that could provide additional insight or clarity to the user."

This is called chain of thought prompting.

In [21]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [23]:
retrieval_augmented_qa_pipeline.run_pipeline("What library is this from ?")

{'response': 'The library mentioned in the text is the Folger Shakespeare Library, located in Washington, DC.',
 'context': [('Get even mor e from the Folger\nYou can get your own copy of this text to keep. Purchase a full copy\nto get the text, plus explanatory notes, illustrations, and more.\n Buy a copyFolger Shakespeare Library\nhttps://shakespeare.folger.edu/\nFront\nMatterFrom the Director of the Folger Shakespeare\nLibrary\nTextual Introduction\nSynopsis\nCharacters in the Play\nACT 1Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nACT 2Scene 1\nScene 2\nScene 3\nScene 4\nACT 3Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nScene 6\nScene 7\nACT 4Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nScene 6\nScene 7\nACT 5Scene 1\nScene 2\nScene 3Contents\nMichael Witmore\nDirector, Folger Shakespeare LibraryIt is hard to imagine a world without Shakespeare. Since their\ncomposition four hundred years ago, Shakespeare’s plays and poems\nhave traveled the globe, inviting those who see and read h

In [22]:
retrieval_augmented_qa_pipeline.run_pipeline("Is there a queen? if so who is it")

{'response': 'There is a mention of a queen in the context provided. The Queen referred to is not specified by name, but she is described as being moved to tears by the contents of the letters she received.',
 'context': [(' her,\n Or cease your quest of love?\n   Most royal Majesty,\n I crave no more than hath your Highness offered,\n Nor will you tender less.\n   Right noble Burgundy,\n When she was dear to us, we did hold her so,\n But now her price is fallen. Sir, there she stands.\n If aught within that little seeming substance,\n Or all of it, with our displeasure pieced\n And nothing more, may fitly like your Grace,\n She’s there, and she is yours.\n   I know no answer.\n \n Will you, with those infirmities she owes,\n Unfriended, new-adopted to our hate,\n Dowered with our curse and strangered with our\n oath,\n Take her or leave her?\n   Pardon me, royal sir,\n Election makes not up in such conditions.\n \n Then leave her, sir, for by the power that made me\n I tell you all he

## Task 6: Visibility Tooling

This is great, but what if we wanted to add some visibility to our pipeline?

Let's use Weights and Biases as a visibility tool!

The first thing we'll need to do is create a Weights and Biases account and get an API key. 

You can follow the process outlined [here](https://docs.wandb.ai/quickstart) to do exactly that!

Now we can get the Weights and Biases dependency and add our key to our env. to begin!

In [24]:
!pip install -q -U wandb

In [25]:
wandb_key = getpass("Weights and Biases API Key: ")
os.environ["WANDB_API_KEY"] = wandb_key

In [26]:
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = "./Pythonic RAG Assignment.ipynb"
wandb.init(project="Visibility Example Customized")

wandb: Currently logged in as: bullyellis. Use `wandb login --relogin` to force relogin


Now we can integrate Weights and Biases into our `RetrievalAugmentedQAPipeline`.

```python
if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
```

The main things to consider here are how to populate the various fields to make sure we're tracking useful information. 

We'll use the `text_only` flag to ensure we can get detailed information about our LLM call!

You can check out all the parameters for Weights and Biases `Trace` [here](https://github.com/wandb/wandb/blob/653015a014281f45770aaf43627f64d9c4f04a32/wandb/sdk/data_types/trace_tree.py#L166)

In [27]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace

class RetrievalAugmentedGenerationPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        
        start_time = datetime.datetime.now().timestamp() * 1000

        try:
            openai_response = self.llm.run([formatted_system_prompt, formatted_user_prompt], text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""

        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
        
        return {"response" : self.llm.run([formatted_user_prompt, formatted_system_prompt]), "context" : context_list} if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message

In [28]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedGenerationPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example Customized"
)

In [29]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")

{'response': "I don't know.",
 'context': [(' you are hid.\n You have now the good advantage of the night.\n Have you not spoken ’gainst the Duke of Cornwall?\n He’s coming hither, now, i’ th’ night, i’ th’ haste,\n And Regan with him. Have you nothing said\n Upon his party ’gainst the Duke of Albany?\n Advise yourself.\n   I am sure on ’t, not a word.\n \n I hear my father coming. Pardon me.\n In cunning I must draw my sword upon you.\n Draw. Seem to defend yourself. Now, quit you\n well. \n Yield! Come before my father! Light, hoa, here!\n  Aside to Edgar. Fly, brother.—Torches, torches!\n —So, farewell. \n Some blood drawn on me would beget opinion\n Of my more fierce endeavor. I have seen drunkards\n Do more than this in sport. \n Father, father!\n Stop, stop! No help?\nEnter Gloucester, and Servants with torches.\n   Now, Edmund, where’s the\n villain?\n \n Here stood he in the dark, his sharp sword out,\n Mumbling of wicked charms, conjuring the moon\n To stand auspicious mistres

In [30]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia?")

{'response': 'Cordelia is tragically hanged in prison, with the blame being laid upon her own despair. Lear, in despair, laments her death, expressing his profound grief and sorrow.',
 'context': [('e.\n Beyond all manner of so much I love you.\n, aside \n What shall Cordelia speak? Love, and be silent.\n, pointing to the map \n Of all these bounds, even from this line to this,\n With shadowy forests and with champains riched,\n With plenteous rivers and wide-skirted meads,\n We make thee lady. To thine and Albany’s issue\n Be this perpetual.—What says our second\n daughter,\n Our dearest Regan, wife of Cornwall? Speak.\n \n I am made of that self mettle as my sister\n And prize me at her worth. In my true heart\n I find she names my very deed of love;\n Only she comes too short, that I profess\n Myself an enemy to all other joys\n Which the most precious square of sense\n possesses,\n And find I am alone felicitate\n In your dear Highness’ love.\n, aside   Then poor Cordelia!\n And ye

Navigate to the Weights and Biases "run" link to see how your LLM is performing!

```
View run at YOUR LINK HERE
```

#### ❓QUESTION:

What is the `model_name` from the WandB `root_span` trace?


#### ANSWER:
gpt-3.5-turbo-0125

## Task 7: RAG Evaluation Using GPT-4



In [31]:
query = "Who is Cordelia?"

response = retrieval_augmented_qa_pipeline.run_pipeline(query)

print(response["response"])

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response and the provided context
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Context: {context}
Response: {response}"""

try:
    chat_openai = ChatOpenAI(model_name="gpt-4-turbo")
except:
    chat_openai = ChatOpenAI()

evaluator_system_prompt = SystemRolePrompt(evaluator_system_template)
evaluation_prompt = UserRolePrompt(evaluation_template)

messages = [
    evaluator_system_prompt.create_message(format=False),
    evaluation_prompt.create_message(
        input=query,
        context="\n".join([context[0] for context in response["context"]]),
        response=response["response"]
    ),
]

chat_openai.run(messages, response_format={"type" : "json_object"})

Cordelia is one of the daughters of King Lear in the play. She is described as being made of the same mettle as her sister and values herself highly. She expresses her love for the king but believes her love is more substantial than words can express. Despite her genuine love, she is ultimately mistreated by her father and sisters.


'{"clarity": "9", "faithfulness": "9", "correctness": "9"}'

# Conclusion

In this notebook, we've gone through the steps required to create your own simple RAQA application!

Please feel free to extend this as much as you'd like. 

In [32]:
wandb.finish()